In [8]:
from cilia_test_mod import gen_system
from cilia_test_mod import d_core
from cilia_test_mod import flow_map
from cilia_test_mod import aug_residual
from cilia_test_mod import b_residual


from scipy import optimize
from scipy.integrate import solve_ivp

from cilia_test_mod import Cilium
from cilia_test_mod import Cilia_System
import numpy as np
from itertools import product

from math import sqrt
from joblib import Parallel, delayed

In [10]:
def symbolic_dynamics(array):

  pd= np.diff(array[3::3])% (2*np.pi)

  rad= array[2::3]

  return rad, np.round(pd, 2), array[0]
  #return ''.join([ 'S' if  5 < d < 2*np.pi else str(int(np.round(d))) for d in pd  ])

def symbols(n_cilias):

  n= n_cilias

  X= [[0, 0.5, np.random.rand()*np.pi*2] for i in range(n)]
  lat=[[2*i,0] for i in range(n)]

  f1 = lambda phi : 1

  forcing= [f1 for i in range(n)]

  X_a=np.append([59.7], X)

  sol = optimize.newton_krylov(lambda X_aug: aug_residual(X_aug
                                                        , lat, forcing, maxstep=1),
                              X_a, verbose= True, maxiter= 50)
  
  return symbolic_dynamics(sol)

def simulate(n_runs, n_cilias):

  return Parallel(n_jobs=4)(delayed(symbols)(n_cilias) for i in range(n_runs))

In [11]:
n= 1

X= [[0, 0.5, 0] for i in range(n)]
lat=[[2*i,0] for i in range(n)]

f1 = lambda phi : 1

forcing= [f1 for i in range(n)]

X_a=np.append([59.7], X)

sol = optimize.newton_krylov(lambda X_aug: aug_residual(X_aug
                                                        , lat, forcing),
                             X_a, verbose= True, maxiter= 50)

sol

0:  |F(x)| = 5.10473e-06; step 1


array([ 5.97797957e+01,  0.00000000e+00,  4.99711087e-01, -7.18917547e-06])

In [12]:
def aug_residual(X_aug, lattice, forcings, cons= [0.5, 0.1 ,10], max_s= 0.1):
    
    '''Augmented residual to find periodic orbit'''
    
    t, X1 = X_aug[0] , X_aug[1:]
    n= int(len(X1)/3)
    
    res= np.append([t], flow_map(t, X1, lattice, forcings, cons, maxstep= max_s))- np.array(X_aug)
    
    res[0]= b_residual(t,X1, lattice, forcings, cons)
    
    for i in range(n):
        res[3*i+3]= min(np.abs(res[3*i+3] % (2*np.pi)), np.abs(2*np.pi - res[3*i+3] % (2*np.pi)))   
        
    return res

In [6]:
aug_residual(X_a, lat, forcing)

array([ 0.        ,  0.        , -0.00244432,  0.00406028,  0.        ,
        0.00188026,  0.02471626])

In [13]:
n= 2

X= [[0, 0.5, 0] for i in range(n)]
lat=[[2*i,0] for i in range(n)]

f1 = lambda phi : 1

forcing= [f1 for i in range(n)]

X_a=np.append([59.7], X)

sol = optimize.newton_krylov(lambda X_aug: aug_residual(X_aug
                                                        , lat, forcing, max_s=1),
                             X_a, verbose= True, maxiter= 50)

sol    

0:  |F(x)| = 0.00228708; step 1
1:  |F(x)| = 1.02099e-05; step 1
2:  |F(x)| = 4.45208e-06; step 1


array([59.67508517,  0.        ,  0.4990042 ,  0.23094023,  0.        ,
        0.49900787, -0.23099086])

In [8]:
n= 2

X= [[0, 0.5, 3*i] for i in range(n)]
lat=[[2*i,0] for i in range(n)]

f1 = lambda phi : 1

forcing= [f1 for i in range(n)]

X_a=np.append([59.7], X)

sol = optimize.newton_krylov(lambda X_aug: aug_residual(X_aug
                                                        , lat, forcing, maxstep=1),
                             X_a, verbose= True, maxiter= 50)

sol

0:  |F(x)| = 0.0250901; step 1
1:  |F(x)| = 7.90473e-05; step 1
2:  |F(x)| = 5.68694e-10; step 1


array([59.89962637,  0.        ,  0.50006622, -1.29246171,  0.        ,
        0.50005149,  1.29242633])

In [ ]:
flow_map(60, X, lat, forcing)

lat= [ [2*i, 2*j] for i, j in product(range(x_n), range(y_n))]
X= [[0, 0.5,0] for i, j in product(range(x_n), range(y_n))]
forcing= [f for i, j in product(range(x_n), range(y_n))]

gen_system(X, lat, forcing, const= [0.5, 0.1 ,10]);

In [10]:
array=np. array([59.89962637,  0.        ,  0.50006622, -1.29246171,  0.        ,
        0.50005149,  1.29242633])

array[3::3]

array([-1.29246171,  1.29242633])

In [7]:
array[1:]
n=2

forcing= [f for i in range(n)]
lat=[[2*i,0] for i in range(n)]

array([ 0.        ,  0.50006622, -1.29246171,  0.        ,  0.50005149,
        1.29242633])

In [12]:
flow_map(array[0], array[1:], lat, forcing) - array[1:]

array([ 0.00000000e+00, -9.45896250e-10,  6.28318526e+00,  0.00000000e+00,
        1.03703712e-10,  1.06593929e-08])

In [ ]:
def perturb(periodic_state, eps= 0.01):
    
    n= len(periodic_state)//3
    
    period, state= periodic_state[0], periodic_state[1:]
    
    f = lambda phi : 1
    
    forcing= [f for i in range(n)]
    lat=[[2*i,0] for i in range(n)]
        
    diff_list=[]

In [4]:
def perturb(periodic_state, eps= 0.001):
    
    n= len(periodic_state)//3
        
    period, state= periodic_state[0], periodic_state[1:]
    
    f = lambda phi : 1
    
    forcing= [f for i in range(n)]
    lat=[[2*i,0] for i in range(n)]
        
    diff_list=[]
        
    mod= np.array(periodic_state) * 0 
    
    for i in range(n):
        
        mod_state= state.copy()
        mod_state[2+ 3*i]+= eps
        
        t=[]
        
        for time in times:
        
            diff= flow_map(time , mod_state, lat, 
                           forcing, maxstep= 5e-6) - flow_map(time,state, lat, forcing, maxstep= 5e-6)
            pd= np.array(diff[2::3])%(2*np.pi) 
            pd_mod=[ pp if pp < np.pi else -2*np.pi+ pp for pp in pd ]
            
            norm= np.linalg.norm(pd_mod)
            
            print(norm)
            
            t.append(norm)
            
        diff_list.append(t)
                
    return diff_list

In [ ]:
def perturb(periodic_state, eps= 0.001):
    
    n= len(periodic_state)//3
        
    period, state= periodic_state[0], periodic_state[1:]
    
    f = lambda phi : 1
    
    forcing= [f for i in range(n)]
    lat=[[2*i,0] for i in range(n)]
        
    diff_list=[]
    
    times= np.linspace(0, 0.1,6)
    
    mod= np.array(periodic_state) * 0 
    
    for i in range(n):
        
        mod_state= state.copy()
        mod_state[2+ 3*i]+= eps
        
        t=[]
        
        for time in times:
        
            diff= flow_map(period, mod_state, lat, 
                           forcing) - flow_map(time,state, lat, forcing, maxstep= 5e-6)
            pd= np.array(diff[2::3])%(2*np.pi) 
            pd_mod=[ pp if pp < np.pi else -2*np.pi+ pp for pp in pd ]
            
            norm= np.linalg.norm(pd_mod)
            
            print(norm)
            
            t.append(norm)
            
        diff_list.append(t)
                
    return diff_list

0.09999999999999964
0.10000147004745955
0.10000294300395529
0.10000441884522183
0.10000589754694253
0.10000737908475281
0.10000000000000009
0.09999861254238619
0.09999722912553546
0.09999584977113282


In [94]:
arr2= [
  0.0010000143635426914,
  0.0010000287616796024,
  0.0010000431941614902,
  0.00100005766076187,
  0.0010000721612489283]

[np.log(i/0.001)/(count+1)/0.02 for count, i in enumerate(arr2)]

[0.0007181719768386505,
 0.0007190316498310522,
 0.0007198871441550487,
 0.0007207387444031533,
 0.0007215864543060547]

In [106]:
array= np.array([59.89962637,  0.        ,  0.50006622, -1.29246171,  0.        ,
        0.50005149,  1.29242633])

perturb(array, eps= -0.01)

0.010000000000000675
0.010000143197556738
0.010000286746959413


KeyboardInterrupt: 

In [113]:
array= np.array([59.67508519,  0.        ,  0.4990042 ,  0.23094024,  0.        ,
        0.49900786, -0.23099086])

perturb(array, eps= 0.1)

0.1
0.09999987650006166
0.09999974049240533
0.09999959196567754
0.0999994309087582
0.0999992573107662
0.10000000000000053
0.09999978445516557
0.09999955717662255
0.09999931818260874
0.0999990674915569
0.09999880512208205


[[0.1,
  0.09999987650006166,
  0.09999974049240533,
  0.09999959196567754,
  0.0999994309087582,
  0.0999992573107662],
 [0.10000000000000053,
  0.09999978445516557,
  0.09999955717662255,
  0.09999931818260874,
  0.0999990674915569,
  0.09999880512208205]]

In [ ]:
perturb(array, eps= -0.001)